<a href="https://colab.research.google.com/github/Martishamao/Sales_predictions/blob/main/Sales_Prediction_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 - Part 5 (Core)
- Martisha Owens
- 9/15/2022



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

filename = '/content/drive/MyDrive/Coding Dojo/05 Intro to Machine Learning/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [15]:
dfc = df.copy()

##1. Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.

In [16]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [25]:
# Are there duplicates?
dfc.duplicated().sum()

0

> This shows that there are no duplicated data.


In [18]:
# Identify missing values.
dfc.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

> Item_Weight has 1463 missing data while Outlet_Size has 2410 missing data.

In [26]:
# Observing the groupings for the Outlet_Size data.
dfc.groupby(['Outlet_Type','Outlet_Location_Type'])['Outlet_Size'].unique()

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 1                          [Small]
                   Tier 3                            [nan]
Supermarket Type1  Tier 1                  [Medium, Small]
                   Tier 2                     [nan, Small]
                   Tier 3                          [Large]
Supermarket Type2  Tier 3                         [Medium]
Supermarket Type3  Tier 3                         [Medium]
Name: Outlet_Size, dtype: object

> Grocery Store only has Outlet_Size data for Tier 1 which is small. Using assumption, Grocery Store, Tier 3 would be Medium or Large.
>For Supermarket Type1, Tier 2, the Outlet_Size is Small. The missing data can be replaced with Small.

In [27]:
# Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent) 
dtypes = dfc.dtypes
dfc_obj = dtypes[dtypes=='object'].index
for i in dfc_obj:
  print(i)
  print(dfc[i].value_counts(dropna=False))
  print('\n\n')

Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



Item_Fat_Content
Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64



Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtyp

Listed are the category and items that needs to be fixed.
- Item_Fat_Content = ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
- Outlet_Size = ['Medium' 'Large' 'High' 'Small']

In [28]:
dfc['Item_Fat_Content'].replace(['low fat', 'LF', 'reg'], ['Low Fat', 'Low Fat','Regular'], inplace = True)
dfc['Outlet_Size'].replace('High', 'Large', inplace = True)
print(dfc['Item_Fat_Content'].unique())
print(dfc['Outlet_Size'].unique())

['Low Fat' 'Regular']
['Medium' nan 'Large' 'Small']


In [29]:
# For any numerical columns, obtain the summary statistics of each (min, max, mean)
dfc.describe(include=[np.number])

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


##2. Identify the features (X) and target (y).

In [30]:
X = dfc.drop(columns = 'Item_Outlet_Sales')
y = dfc['Item_Outlet_Sales']

## 3. Perform a train test split 


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer

from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [33]:
X_perc = len(X_train)/(len(X_train)+len(X_test))*100
y_perc = len(y_train)/(len(y_train)+len(y_test))*100
print(f'len of X train:{len(X_train)}, X test:{len(X_test)}, y train:{len(y_train)}, y test:{len(y_test)}.')
print(f'Split Ratio X:{round(X_perc,1)}% and y:{round(y_perc,1)}%')

len of X train:6392, X test:2131, y train:6392, y test:2131.
Split Ratio X:75.0% and y:75.0%


## 4. Create a preprocessing object to prepare the dataset for Machine Learning.

In [40]:
obj_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
scaler = StandardScaler()

##5. Make sure your imputation of missing values occurs  after the train test split using SimpleImputer.

In [41]:
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

In [42]:
obj_pipe = make_pipeline(freq_imputer, ohe)
obj_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [43]:
num_pipe = make_pipeline(mean_imputer, scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

###All preprocessing steps should be contained within a single preprocessing object.

In [44]:
obj_tuple = (obj_pipe,obj_selector)
num_tuple = (num_pipe,num_selector)

preprocessor = make_column_transformer(obj_tuple,num_tuple)
preprocessor
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f93cb58ef90>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f93cb58e650>)])

###Using preprocessing object to transform the data.

In [45]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [46]:
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('shape of data is', X_train_processed.shape)

0 missing values in training data
0 missing values in testing data
All data in X_train_processed are float64
All data in X_test_processed are float64
shape of data is (6392, 1592)
